In [5]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [6]:
df = pd.read_csv("C:\kelompok-11-Machine-Learning\dataset\heart_failure_clinical_records_dataset.csv")
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\k'
<>:1: SyntaxWarning: invalid escape sequence '\k'
C:\Users\STUDENT\AppData\Local\Temp\ipykernel_15616\1264800891.py:1: SyntaxWarning: invalid escape sequence '\k'
  df = pd.read_csv("C:\kelompok-11-Machine-Learning\dataset\heart_failure_clinical_records_dataset.csv")


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [17]:
X = df.drop("DEATH_EVENT", axis=1)
y = df["DEATH_EVENT"]


In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [10]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])


c:\Users\STUDENT\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [12]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.5131 - loss: 0.6849 - val_accuracy: 0.5833 - val_loss: 0.6515
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6387 - loss: 0.6525 - val_accuracy: 0.6667 - val_loss: 0.6111
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7016 - loss: 0.6170 - val_accuracy: 0.6875 - val_loss: 0.5900
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7225 - loss: 0.5571 - val_accuracy: 0.7083 - val_loss: 0.5689
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7749 - loss: 0.5358 - val_accuracy: 0.6875 - val_loss: 0.5552
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6963 - loss: 0.5432 - val_accuracy: 0.6875 - val_loss: 0.5430
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7801 - loss: 0.4952 - val_accuracy: 0.6875 - val_loss: 0.5335
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7958 - loss: 0.4797 - val_accuracy: 0.

In [13]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


In [14]:
confusion_matrix(y_test, y_pred)


array([[40,  1],
       [ 8, 11]])

In [15]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.98      0.90        41
           1       0.92      0.58      0.71        19

    accuracy                           0.85        60
   macro avg       0.88      0.78      0.80        60
weighted avg       0.86      0.85      0.84        60



In [16]:
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC-AUC Score:", roc_auc)


ROC-AUC Score: 0.8934531450577664
